# Met Eireann wind speed analysis
### Programming for Data Analytics Project
Author: Vanessa de Oliveira Lyra


Location: Grousemount Wind Farm  
Capacity: 114 MW (38 turbines) — one of the largest in Cork and among the biggest onshore windfarms in Ireland.  

Met Éireann Weather Stations close to Grousemount:  
- Cork Airport Synoptic Station (Co. Cork)  
- Moorepark (Fermoy) Weather Station (Co. Cork)  
- Sherkin Island Weather Station (Co. Cork)  
- Valentia Observatory (Co. Kerry) 

Grousemount Wind Farm info: https://esb.ie/news---insights/blog/article/esb/2020/04/30/grousemount-wind-farm-bringing-power-to-communities-across-ireland-with-wind-energy  
Weather stations and wind data: https://www.met.ie/climate/available-data/historical-data

Importing libraries

In [41]:
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3

Database connection and data fetching

In [42]:
# Connecting to SQLite and creating database file
conn = sqlite3.connect("windspeed.db")

# Reading .csv file into a dataframe using semicolon separator
df_cork = pd.read_csv("datasets/dly3904_cork.csv",skiprows=24)
df_moore = pd.read_csv("datasets/dly575_moore.csv",skiprows=24)
df_sherkin = pd.read_csv("datasets/dly775_sherkin.csv",skiprows=24)
df_valentia = pd.read_csv("datasets/dly2275_valentia.csv",skiprows=24)

# Check first 5 lines of data for reference
df_cork.head()

,date,ind,maxtp,ind.1,mintp,igmin,gmin,ind.2,rain,cbl,...,ind.5,hg,sun,dos,soil,pe,evap,smd_wd,smd_md,smd_pd
0,01-jan-1962,0,2.8,1,-2.5,,,8,,999.6,...,0,24,,0,,,,,,
1,02-jan-1962,0,2.8,1,-3.7,,,8,,1005.8,...,0,10,,0,,,,,,
2,03-jan-1962,0,2.3,1,-3.8,,,8,,1007.8,...,0,13,,0,,,,,,
3,04-jan-1962,0,6.6,0,0.6,,,8,,1007.8,...,0,12,,0,,,,,,
4,05-jan-1962,0,9.3,0,0.6,,,8,,1002.2,...,0,29,,0,,,,,,


##### References:
Skiprows function: https://stackoverflow.com/questions/20637439/skip-rows-during-csv-import-pandas   
Writing data to SQlite: https://stackoverflow.com/questions/14431646/how-to-write-pandas-dataframe-to-sqlite-with-index  

In [43]:
# Print headers for reference
# headers = df.columns.tolist()
# headers

Cleaning up dataset data

In [44]:
# Cork dataset

# Drop unwanted columns
drop_columns = ["ind","maxtp","ind.1","mintp","igmin","gmin","ind.2","rain","cbl","ind.3","ind.4","ind.5","sun","dos","soil","pe","evap","smd_wd","smd_md","smd_pd"]
df_cork.drop(columns=drop_columns, inplace=True, errors='ignore')

# Normalize date column
# Remove whitespaces from date
df_cork["date"] = df_cork["date"].astype(str).str.strip()

# Convert date to Datetime and define format
df_cork["date"] = pd.to_datetime(df_cork["date"], format="%d-%b-%Y", errors="coerce")

# Normalize other columns
# Convert mean wind Speed and wind direction and highest gust to numeric values
df_cork["wdsp"] = pd.to_numeric(df_cork["wdsp"], errors="coerce")   
df_cork["hm"] = pd.to_numeric(df_cork["hm"], errors="coerce") 
df_cork["hg"] = pd.to_numeric(df_cork["hg"], errors="coerce")
df_cork["ddhm"] = pd.to_numeric(df_cork["ddhm"], errors="coerce")

# Checking dataframe after clean up for testing
# df_cork.head()

# Writing datafame to database
df_cork.to_sql("cork_data", conn, if_exists="replace", index=False)

23345

In [45]:
# Moore dataset

# Drop unwanted columns
drop_columns = ["ind","maxtp","ind.1","mintp","igmin","gmin","ind.2","rain","cbl","ind.3","ind.4","ind.5","sun","dos","soil","pe","evap","smd_wd","smd_md","smd_pd"]
df_moore.drop(columns=drop_columns, inplace=True, errors='ignore')

# Normalize date column
# Remove whitespaces from date
df_moore["date"] = df_moore["date"].astype(str).str.strip()

# Convert date to Datetime and define format
df_moore["date"] = pd.to_datetime(df_moore["date"], format="%d-%b-%Y", errors="coerce")

# Normalize other columns
# Convert mean wind Speed and wind direction and highest gust to numeric values
df_moore["wdsp"] = pd.to_numeric(df_moore["wdsp"], errors="coerce")   
df_moore["hm"] = pd.to_numeric(df_moore["hm"], errors="coerce") 
df_moore["hg"] = pd.to_numeric(df_moore["hg"], errors="coerce")
df_moore["ddhm"] = pd.to_numeric(df_moore["ddhm"], errors="coerce")

# Writing datafame to database
df_moore.to_sql("moore_data", conn, if_exists="replace", index=False)

8084

In [46]:
# Sherkin dataset

# Drop unwanted columns
drop_columns = ["ind","maxtp","ind.1","mintp","igmin","gmin","ind.2","rain","cbl","ind.3","ind.4","ind.5","sun","dos","soil","pe","evap","smd_wd","smd_md","smd_pd"]
df_sherkin.drop(columns=drop_columns, inplace=True, errors='ignore')

# Normalize date column
# Remove whitespaces from date
df_sherkin["date"] = df_sherkin["date"].astype(str).str.strip()

# Convert date to Datetime and define format
df_sherkin["date"] = pd.to_datetime(df_sherkin["date"], format="%d-%b-%Y", errors="coerce")

# Normalize other columns
# Convert mean wind Speed and wind direction and highest gust to numeric values
df_sherkin["wdsp"] = pd.to_numeric(df_sherkin["wdsp"], errors="coerce")   
df_sherkin["hm"] = pd.to_numeric(df_sherkin["hm"], errors="coerce") 
df_sherkin["hg"] = pd.to_numeric(df_sherkin["hg"], errors="coerce")
df_sherkin["ddhm"] = pd.to_numeric(df_sherkin["ddhm"], errors="coerce")

# Writing datafame to database
df_sherkin.to_sql("sherkin_data", conn, if_exists="replace", index=False)

7869

In [47]:
# Valentia dataset

# Drop unwanted columns
drop_columns = ["ind","maxtp","ind.1","mintp","igmin","gmin","ind.2","rain","cbl","ind.3","ind.4","ind.5","sun","dos","soil","pe","evap","smd_wd","smd_md","smd_pd"]
df_valentia.drop(columns=drop_columns, inplace=True, errors='ignore')

# Normalize date column
# Remove whitespaces from date
df_valentia["date"] = df_valentia["date"].astype(str).str.strip()

# Convert date to Datetime and define format
df_valentia["date"] = pd.to_datetime(df_valentia["date"], format="%d-%b-%Y", errors="coerce")

# Normalize other columns
# Convert mean wind Speed and wind direction and highest gust to numeric values
df_valentia["wdsp"] = pd.to_numeric(df_valentia["wdsp"], errors="coerce")   
df_valentia["hm"] = pd.to_numeric(df_valentia["hm"], errors="coerce") 
df_valentia["hg"] = pd.to_numeric(df_valentia["hg"], errors="coerce")
df_valentia["ddhm"] = pd.to_numeric(df_valentia["ddhm"], errors="coerce")

# Writing datafame to database
df_valentia.to_sql("valentia_data", conn, if_exists="replace", index=False)

30649

##### References
Remove whitespaces from date: https://medium.com/@amit25173/how-to-remove-whitespace-from-strings-in-pandas-bfd9acdc55f3  
Converting datetime: https://pbpython.com/pandas_dtypes.html  
Convert data to numeric values https://kajodata.com/en/knowledge-base-excel-sql-python/knowledge-base-python-tech-skillshow-pandas-to_numeric-works-in-python-examples-mmk/  
Dates showing as NaT after claening data updated date format from %d-%b-%y to %d-%b-%Y to handle 4-digits year: https://medium.com/@codecorecomputercenter/python-date-formats-a-comprehensive-guide-c6ed0224980b     

In [48]:
# Group data by year and calculate mean
#yearly_wdsp = df.groupby(df["date"].dt.year)["wdsp"].mean()

Group by year
https://blog.finxter.com/5-best-ways-to-group-pandas-dataframe-by-year/

Yearly mean wind speed